In [8]:
import sounddevice as sd
import librosa
import numpy as np
import scipy.io.wavfile as wav
import os
from sklearn.metrics.pairwise import cosine_similarity

def record_voice(filename="temp.wav", duration=3, fs=16000):
    print("Recording... Speak now.")
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()
    wav.write(filename, fs, audio)
    print("Recording complete.")

def extract_features(filename):
    y, sr = librosa.load(filename, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfcc.T, axis=0)

def enroll_user(username):
    filename = f"{username}.wav"
    record_voice(filename)
    features = extract_features(filename)
    np.save(f"{username}_voiceprint.npy", features)
    print(f"[+] User '{username}' enrolled successfully.\n")

def authenticate_user(username):
    voiceprint_path = f"{username}_voiceprint.npy"
    if not os.path.exists(voiceprint_path):
        print(f"[!] User '{username}' not enrolled. Please enroll first.")
        return

    record_voice("test.wav")
    test_features = extract_features("test.wav")
    stored_features = np.load(voiceprint_path)

    similarity = cosine_similarity([test_features], [stored_features])[0][0]
    print(f"[i] Similarity score: {similarity:.2f}")

    if similarity > 0.85:  # Threshold for successful match
        print("[✔] Authentication successful!\n")
    else:
        print("[✘] Authentication failed. The voice input does not match the registered voice.\n")

def main():
    print("🎙️  Voice-Based Authentication System")
    
    while True:
        choice = input("Choose: [1] Enroll  [2] Authenticate: ")

        if choice == "1":
            username = input("Enter username: ").strip().lower()
            enroll_user(username)
            break
        elif choice == "2":
            username = input("Enter username: ").strip().lower()
            authenticate_user(username)
            break
        else:
            print("Invalid choice, please enter '1' to enroll or '2' to authenticate.")

if __name__ == "__main__":
    main()


🎙️  Voice-Based Authentication System


Choose: [1] Enroll  [2] Authenticate:  2
Enter username:  alice


Recording... Speak now.
Recording complete.
[i] Similarity score: 1.00
[✔] Authentication successful!

